<a href="https://colab.research.google.com/github/fpcaforio/grace/blob/master/Grad_CAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
import tensorflow as tf
import pickle
import numpy as np
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import cv2

In [20]:
#CONFIG

dataset = 'NLS-KDD'
N_CLASSES = 2

if dataset == 'NLS-KDD':
  print('Dataset: NLS-KDD')
  pathMagneto = '/content/drive/MyDrive/Grace/NLS-KDD/magneto/'
  size = 12
elif dataset == 'UNSW-NB15':
  print('Dataset: UNSW-NB15')
  pathMagneto = '/content/drive/MyDrive/Grace/UNSW-NB15/magneto/'
  size = 15
elif dataset == 'KDD-CUP99':  
  print('KDD-CUP99')
  pathMagneto = '/content/drive/MyDrive/Grace/KDD-CUP99/magneto/'
  size = 13

from google.colab import drive
drive.mount('/content/drive')

Dataset: NLS-KDD
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
model = tf.keras.models.load_model(pathMagneto + 'CNN_model.h5')
model.summary()

last_conv_layer_name = 'conv2'

# all layers
classifier_layer_names = [
    'flatten', 'dense', 'dense_1', 'dense_2',
]

with open(pathMagneto + 'train_image.pickle', 'rb') as t:
    XTrain = pickle.load(t)
t.close

with open(pathMagneto + 'test_image.pickle', 'rb') as t:
    XTest = pickle.load(t)
t.close

with open(pathMagneto + 'Ytrain.pickle', 'rb') as k:
    YTrain = pickle.load(k)
k.close

with open(pathMagneto + 'Ytest.pickle', 'rb') as k:
    YTest = pickle.load(k)
k.close

XTrain = np.array(XTrain)
XTest = np.array(XTest)
YTrain = np.array(YTrain)
YTest = np.array(YTest)

print('XTrain: ', XTrain.shape)
print('XTest: ',XTest.shape)
print('YTrain: ',YTrain.shape)
print('YTest: ',YTest.shape)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 12, 12, 3)]       0         
_________________________________________________________________
conv0 (Conv2D)               (None, 9, 9, 32)          1568      
_________________________________________________________________
dropout (Dropout)            (None, 9, 9, 32)          0         
_________________________________________________________________
conv1 (Conv2D)               (None, 6, 6, 64)          32832     
_________________________________________________________________
dropout_1 (Dropout)          (None, 6, 6, 64)          0         
_________________________________________________________________
conv2 (Conv2D)               (None, 3, 3, 128)         131200    
_________________________________________________________________
flatten (Flatten)            (None, 1152)              0     

In [18]:
imgrgb = np.stack((XTest,)*3, axis=-1)
preds = model.predict(imgrgb, verbose=1)
y_pred = np.argmax(preds, axis=1)
con_mat = confusion_matrix(YTest, y_pred)

tp = con_mat[0][0]  # attacks true
fn = con_mat[0][1]  # attacs predict normal
fp = con_mat[1][0]  # normal predict attacks
tn = con_mat[1][1]  # normal as normal
attacks = tp + fn
normals = fp + tn
OA = (tp + tn) / (attacks + normals)
AA = ((tp / attacks) + (tn / normals)) / N_CLASSES
P = tp / (tp + fp)
R = tp / (tp + fn)
F1 = 2 * ((P * R) / (P + R))
FAR = fp / (fp + tn)
TPR = tp / (tp + fn)
r = (tp, fn, fp, tn, OA, AA, P, R, F1, FAR, TPR)

print(r)

705/705 [==============================] - 4s 5ms/step
(9201, 3632, 728, 8983, 0.8066004258339248, 0.8210065645057698, 0.9266794239097593, 0.7169796618093977, 0.808452684298392, 0.0749665327978581, 0.7169796618093977)


In [19]:
def make_gradcam_heatmap(img_array, model, last_conv_layer_name, classifier_layer_names):

    last_conv_layer = model.get_layer(last_conv_layer_name)
    last_conv_layer_model = tf.keras.Model(model.inputs, last_conv_layer.output)

    classifier_input = tf.keras.Input(shape=last_conv_layer.output.shape[1:])
    x = classifier_input
    for layer_name in classifier_layer_names:
        x = model.get_layer(layer_name)(x)
    classifier_model = tf.keras.Model(classifier_input, x)

    with tf.GradientTape() as tape:
        last_conv_layer_output = last_conv_layer_model(img_array)
        tape.watch(last_conv_layer_output)
        preds = classifier_model(last_conv_layer_output)
        top_pred_index = tf.argmax(preds[0])
        top_class_channel = preds[:, top_pred_index]

    grads = tape.gradient(top_class_channel, last_conv_layer_output)

    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    last_conv_layer_output = last_conv_layer_output.numpy()[0]
    pooled_grads = pooled_grads.numpy()

    for i in range(pooled_grads.shape[-1]):  # 128
        last_conv_layer_output[:, :, i] *= pooled_grads[i]

    heatmap = np.mean(last_conv_layer_output, axis=-1)
    heatmap = np.maximum(heatmap, 0) / np.max(heatmap)

    return heatmap

In [ ]:
imgrgb = np.stack((XTrain,)*3, axis=-1)

XTrain_heatmap = []

for i in range (0, len(XTrain)):
  
  print('Istanza #: ', i)

  array = np.expand_dims(imgrgb[i], axis=0)
  heatmap = make_gradcam_heatmap(array, model, last_conv_layer_name, classifier_layer_names)
  heatmap = cv2.resize(heatmap, (size, size))
  #plt.imshow(heatmap)
  #plt.show()
  XTrain_heatmap.append(heatmap)

#f_myfile = open(pathSave, 'wb') 
#pickle.dump(XTrain_heatmap, f_myfile)
#f_myfile.close()

In [ ]:
imgrgb = np.stack((XTest,)*3, axis=-1)

XTest_heatmap = []

for i in range (0, len(XTest)):

  print('Istanza #: ', i)

  array = np.expand_dims(imgrgb[i], axis=0)
  heatmap = make_gradcam_heatmap(array, model, last_conv_layer_name, classifier_layer_names)
  heatmap = cv2.resize(heatmap, (size, size))
  #plt.imshow(heatmap)
  #plt.show()
  XTest_heatmap.append(heatmap)

#f_myfile = open(pathSave, 'wb') 
#pickle.dump(XTrain_heatmap, f_myfile)
#f_myfile.close()